In [1]:
import numpy as np
from pathlib import Path
from mido import MidiFile, MidiTrack, Message, MetaMessage
import copy
from typing import List, Dict, Tuple
import os

## Configuration Constants

In [2]:
# Paths
INPUT_DIR = Path('../dataset/midi_files/mpe')
OUTPUT_BASE_DIR = Path('../dataset/midi_files/mpe53')

# 53-EDO constants
STEPS_PER_OCTAVE = 53
CENTS_PER_STEP = 1200 / STEPS_PER_OCTAVE  # ≈ 22.64 cents

# MPE Pitch Bend Configuration
# Standard MIDI pitch bend range: ±2 semitones (can be configured via RPN)
PITCH_BEND_RANGE_SEMITONES = 2
PITCH_BEND_MAX = 8191  # Maximum positive pitch bend value

print(f"53-EDO step size: {CENTS_PER_STEP:.4f} cents")
print(f"Input directory: {INPUT_DIR}")
print(f"Output base directory: {OUTPUT_BASE_DIR}")

53-EDO step size: 22.6415 cents
Input directory: ../dataset/midi_files/mpe
Output base directory: ../dataset/midi_files/mpe53


## Core Functions: Pattern to Chromatic Mapping

In [3]:
def pattern_to_diatonic_mapping(pattern: List[int]) -> Dict[int, int]:
    """
    Convert a 7-element diatonic pattern to diatonic degree → 53-EDO step mapping.
    
    Args:
        pattern: List of 7 integers representing intervals between diatonic notes.
                 Example: [9, 9, 4, 9, 9, 9, 4] for Pythagorean-like tuning.
    
    Returns:
        Dictionary mapping diatonic degrees (0-6) to 53-EDO steps (0-52).
        0=C, 1=D, 2=E, 3=F, 4=G, 5=A, 6=B
    """
    if len(pattern) != 7:
        raise ValueError(f"Pattern must have exactly 7 elements, got {len(pattern)}")
    
    if sum(pattern) != 53:
        raise ValueError(f"Pattern must sum to 53, got {sum(pattern)}")
    
    # Build cumulative steps: C=0, D=pattern[0], E=pattern[0]+pattern[1], etc.
    diatonic_steps = {0: 0}  # C = 0
    cumulative = 0
    
    for i, interval in enumerate(pattern[:-1]):  # Don't need last interval (B→C)
        cumulative += interval
        diatonic_steps[i + 1] = cumulative
    
    return diatonic_steps


def interpolate_chromatic_mapping(pattern: List[int], sharp_bias: float = 0.5) -> Dict[int, int]:
    """
    Interpolate chromatic pitch classes (0-11) to 53-EDO steps based on diatonic pattern.
    
    The chromatic notes (sharps/flats) are calculated by interpolating within
    the intervals between diatonic notes.
    
    Args:
        pattern: 7-element diatonic interval pattern in 53-EDO steps.
        sharp_bias: Where to place accidentals within the interval (0.0-1.0).
                    0.5 = middle (default), lower = closer to natural below,
                    higher = closer to natural above.
    
    Returns:
        Dictionary mapping 12-TET pitch classes (0-11) to 53-EDO steps (0-52).
        
    12-TET pitch classes:
        0=C, 1=C#/Db, 2=D, 3=D#/Eb, 4=E, 5=F, 6=F#/Gb, 7=G, 8=G#/Ab, 9=A, 10=A#/Bb, 11=B
    """
    # Get diatonic mapping first
    diatonic = pattern_to_diatonic_mapping(pattern)
    
    # Map diatonic degrees to 12-TET pitch classes
    # Diatonic: 0=C, 1=D, 2=E, 3=F, 4=G, 5=A, 6=B
    # 12-TET:   0=C, 2=D, 4=E, 5=F, 7=G, 9=A, 11=B
    diatonic_to_12tet = {0: 0, 1: 2, 2: 4, 3: 5, 4: 7, 5: 9, 6: 11}
    
    # Start with diatonic notes
    chromatic_mapping = {}
    for deg, pc in diatonic_to_12tet.items():
        chromatic_mapping[pc] = diatonic[deg]
    
    # Interpolate chromatic notes (accidentals)
    # Each accidental sits between two diatonic notes
    accidentals = [
        (1, 0, 1, pattern[0]),   # C# between C(0) and D(1), interval = pattern[0]
        (3, 1, 2, pattern[1]),   # D# between D(1) and E(2), interval = pattern[1]
        # Note: No accidental between E and F (semitone in 12-TET)
        (6, 3, 4, pattern[3]),   # F# between F(3) and G(4), interval = pattern[3]
        (8, 4, 5, pattern[4]),   # G# between G(4) and A(5), interval = pattern[4]
        (10, 5, 6, pattern[5]),  # A# between A(5) and B(6), interval = pattern[5]
        # Note: No accidental between B and C (semitone in 12-TET)
    ]
    
    for pc, lower_deg, upper_deg, interval in accidentals:
        lower_step = diatonic[lower_deg]
        # Calculate position within interval based on bias
        offset = round(interval * sharp_bias)
        chromatic_mapping[pc] = lower_step + offset
    
    return chromatic_mapping


# Test the mapping functions
test_pattern = [9, 9, 4, 9, 9, 9, 4]  # Pythagorean-like
diatonic_map = pattern_to_diatonic_mapping(test_pattern)
chromatic_map = interpolate_chromatic_mapping(test_pattern)

print("Diatonic mapping (degree → 53-EDO step):")
degree_names = ['C', 'D', 'E', 'F', 'G', 'A', 'B']
for deg, step in diatonic_map.items():
    print(f"  {degree_names[deg]}: {step}")

print("\nChromatic mapping (pitch class → 53-EDO step):")
pc_names = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
for pc in range(12):
    print(f"  {pc_names[pc]:>2} ({pc:>2}): {chromatic_map[pc]:>2} steps")

Diatonic mapping (degree → 53-EDO step):
  C: 0
  D: 9
  E: 18
  F: 22
  G: 31
  A: 40
  B: 49

Chromatic mapping (pitch class → 53-EDO step):
   C ( 0):  0 steps
  C# ( 1):  4 steps
   D ( 2):  9 steps
  D# ( 3): 13 steps
   E ( 4): 18 steps
   F ( 5): 22 steps
  F# ( 6): 26 steps
   G ( 7): 31 steps
  G# ( 8): 35 steps
   A ( 9): 40 steps
  A# (10): 44 steps
   B (11): 49 steps


## MPE Conversion Functions

In [4]:
def midi_note_to_53edo(midi_note: int, chromatic_mapping: Dict[int, int]) -> Tuple[int, float]:
    """
    Convert a MIDI note number to 53-EDO representation.
    
    Args:
        midi_note: MIDI note number (0-127)
        chromatic_mapping: Pitch class (0-11) → 53-EDO step mapping
    
    Returns:
        Tuple of (base_midi_note, pitch_bend_semitones)
        - base_midi_note: The MIDI note to play (same as input for MPE)
        - pitch_bend_semitones: Pitch bend in semitones to achieve 53-EDO tuning
    """
    # Extract pitch class and octave
    pitch_class = midi_note % 12
    octave = midi_note // 12
    
    # Get 53-EDO step for this pitch class
    step_in_octave = chromatic_mapping[pitch_class]
    
    # Calculate the target frequency ratio relative to 12-TET
    # 12-TET: pitch_class semitones = pitch_class * 100 cents
    # 53-EDO: step_in_octave steps = step_in_octave * (1200/53) cents
    
    cents_12tet = pitch_class * 100
    cents_53edo = step_in_octave * CENTS_PER_STEP
    
    # Difference in cents (how much to bend)
    cents_deviation = cents_53edo - cents_12tet
    
    # Convert cents to semitones
    pitch_bend_semitones = cents_deviation / 100
    
    return midi_note, pitch_bend_semitones


def semitones_to_pitch_bend(semitones: float, bend_range: int = PITCH_BEND_RANGE_SEMITONES) -> int:
    """
    Convert semitones deviation to MIDI pitch bend value.
    
    Args:
        semitones: Pitch deviation in semitones
        bend_range: Pitch bend range in semitones (typically ±2)
    
    Returns:
        MIDI pitch bend value (-8192 to 8191)
    """
    # Normalize to -1..1 range based on bend range
    normalized = semitones / bend_range
    
    # Clamp to valid range
    normalized = max(-1.0, min(1.0, normalized))
    
    # Convert to MIDI pitch bend value
    # -8192 to 8191, with 0 being center (no bend)
    if normalized >= 0:
        return int(normalized * PITCH_BEND_MAX)
    else:
        return int(normalized * 8192)


# Test conversion
print("Testing MIDI note to 53-EDO conversion:")
print(f"{'Note':<6} {'MIDI':<6} {'53-EDO Step':<12} {'Bend (st)':<12} {'Bend (cents)':<12}")
print("-" * 50)

for midi_note in [60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71]:  # C4 to B4
    base, bend_st = midi_note_to_53edo(midi_note, chromatic_map)
    pc = midi_note % 12
    step = chromatic_map[pc]
    print(f"{pc_names[pc]:<6} {midi_note:<6} {step:<12} {bend_st:<12.4f} {bend_st * 100:<12.2f}")

Testing MIDI note to 53-EDO conversion:
Note   MIDI   53-EDO Step  Bend (st)    Bend (cents)
--------------------------------------------------
C      60     0            0.0000       0.00        
C#     61     4            -0.0943      -9.43       
D      62     9            0.0377       3.77        
D#     63     13           -0.0566      -5.66       
E      64     18           0.0755       7.55        
F      65     22           -0.0189      -1.89       
F#     66     26           -0.1132      -11.32      
G      67     31           0.0189       1.89        
G#     68     35           -0.0755      -7.55       
A      69     40           0.0566       5.66        
A#     70     44           -0.0377      -3.77       
B      71     49           0.0943       9.43        


## MIDI File Processing

In [5]:
def convert_midi_to_mpe_53edo(
    input_path: Path,
    output_path: Path,
    chromatic_mapping: Dict[int, int],
    pitch_bend_range: int = PITCH_BEND_RANGE_SEMITONES
) -> bool:
    """
    Convert a standard MIDI file to MPE format with 53-EDO tuning.
    
    MPE assigns each note to a separate MIDI channel (1-15) so that
    pitch bend affects only that note.
    
    Args:
        input_path: Path to input MIDI file
        output_path: Path to save output MPE MIDI file
        chromatic_mapping: Pitch class → 53-EDO step mapping
        pitch_bend_range: Pitch bend range in semitones
    
    Returns:
        True if successful, False otherwise
    """
    try:
        mid = MidiFile(input_path)
    except Exception as e:
        print(f"Error loading {input_path}: {e}")
        return False
    
    # Create new MIDI file
    new_mid = MidiFile(ticks_per_beat=mid.ticks_per_beat)
    
    # MPE channel allocation (channels 1-15, channel 0 reserved for global)
    # We'll use a round-robin allocation for simplicity
    mpe_channels = list(range(1, 16))  # Channels 1-15
    channel_index = 0
    
    # Track active notes: (original_channel, note) -> mpe_channel
    active_notes = {}
    
    for track in mid.tracks:
        new_track = MidiTrack()
        new_mid.tracks.append(new_track)
        
        for msg in track:
            if msg.is_meta:
                # Copy meta messages as-is
                new_track.append(msg.copy())
            
            elif msg.type == 'note_on' and msg.velocity > 0:
                # Calculate 53-EDO tuning
                base_note, bend_semitones = midi_note_to_53edo(msg.note, chromatic_mapping)
                pitch_bend = semitones_to_pitch_bend(bend_semitones, pitch_bend_range)
                
                # Allocate MPE channel
                mpe_ch = mpe_channels[channel_index % len(mpe_channels)]
                channel_index += 1
                
                # Store allocation
                active_notes[(msg.channel, msg.note)] = mpe_ch
                
                # First, send pitch bend (with time delta)
                new_track.append(Message(
                    'pitchwheel',
                    channel=mpe_ch,
                    pitch=pitch_bend,
                    time=msg.time
                ))
                
                # Then, send note on (time=0 since pitch bend already has the delta)
                new_track.append(Message(
                    'note_on',
                    channel=mpe_ch,
                    note=base_note,
                    velocity=msg.velocity,
                    time=0
                ))
            
            elif msg.type == 'note_off' or (msg.type == 'note_on' and msg.velocity == 0):
                # Find the MPE channel for this note
                key = (msg.channel, msg.note)
                if key in active_notes:
                    mpe_ch = active_notes.pop(key)
                    new_track.append(Message(
                        'note_off',
                        channel=mpe_ch,
                        note=msg.note,
                        velocity=msg.velocity,
                        time=msg.time
                    ))
                else:
                    # Note wasn't tracked, pass through
                    new_track.append(msg.copy())
            
            elif msg.type == 'control_change':
                # Copy control changes to all MPE channels or just pass through
                new_track.append(msg.copy())
            
            elif msg.type == 'program_change':
                # Copy program changes
                new_track.append(msg.copy())
            
            elif msg.type == 'pitchwheel':
                # Skip original pitch bend messages (we're generating new ones)
                # But preserve the time delta
                if new_track:
                    # Add the time to the next message or create a dummy
                    pass  # Time will be accumulated
            
            else:
                # Copy other messages
                new_track.append(msg.copy())
    
    # Save output file
    try:
        output_path.parent.mkdir(parents=True, exist_ok=True)
        new_mid.save(output_path)
        return True
    except Exception as e:
        print(f"Error saving {output_path}: {e}")
        return False

## Main Processing Function

In [6]:
def process_dataset_with_pattern(
    pattern: List[int],
    input_dir: Path = INPUT_DIR,
    output_base_dir: Path = OUTPUT_BASE_DIR,
    sharp_bias: float = 0.5,
    verbose: bool = True
) -> Dict[str, any]:
    """
    Process all MIDI files in input directory and convert to 53-EDO using the given pattern.
    
    Args:
        pattern: 7-element list of integers representing diatonic intervals in 53-EDO.
        input_dir: Directory containing input MIDI files.
        output_base_dir: Base directory for output (pattern subfolder will be created).
        sharp_bias: Position of accidentals within intervals (0.0-1.0).
        verbose: Whether to print progress.
    
    Returns:
        Dictionary with processing statistics.
    """
    # Validate pattern
    if len(pattern) != 7 or sum(pattern) != 53:
        raise ValueError(f"Invalid pattern: must have 7 elements summing to 53")
    
    # Create pattern string for directory name
    pattern_str = '-'.join(map(str, pattern))
    output_dir = output_base_dir / pattern_str
    
    if verbose:
        print(f"="*60)
        print(f"53-EDO Data Augmentation")
        print(f"="*60)
        print(f"Pattern: {pattern}")
        print(f"Pattern string: {pattern_str}")
        print(f"Input directory: {input_dir}")
        print(f"Output directory: {output_dir}")
        print(f"Sharp bias: {sharp_bias}")
        print(f"="*60)
    
    # Generate chromatic mapping
    chromatic_mapping = interpolate_chromatic_mapping(pattern, sharp_bias)
    
    if verbose:
        print("\nChromatic mapping (pitch class → 53-EDO step):")
        for pc in range(12):
            cents_diff = (chromatic_mapping[pc] * CENTS_PER_STEP) - (pc * 100)
            print(f"  {pc_names[pc]:>2}: step {chromatic_mapping[pc]:>2} ({cents_diff:+.1f} cents from 12-TET)")
        print()
    
    # Find all MIDI files
    midi_files = list(input_dir.glob('*.mid'))
    
    if not midi_files:
        print(f"No MIDI files found in {input_dir}")
        return {'total': 0, 'success': 0, 'failed': 0, 'pattern': pattern_str}
    
    if verbose:
        print(f"Found {len(midi_files)} MIDI files to process\n")
    
    # Process files
    success_count = 0
    failed_count = 0
    failed_files = []
    
    for i, input_file in enumerate(midi_files):
        # Generate output filename
        # Change prefix from mpe_ to 53TET_
        output_name = input_file.name
        if output_name.startswith('mpe_'):
            output_name = '53TET_' + output_name[4:]
        else:
            output_name = '53TET_' + output_name
        
        output_file = output_dir / output_name
        
        # Convert
        success = convert_midi_to_mpe_53edo(input_file, output_file, chromatic_mapping)
        
        if success:
            success_count += 1
            if verbose and (i + 1) % 100 == 0:
                print(f"Processed {i + 1}/{len(midi_files)} files...")
        else:
            failed_count += 1
            failed_files.append(input_file.name)
    
    # Summary
    if verbose:
        print(f"\n" + "="*60)
        print(f"Processing Complete!")
        print(f"="*60)
        print(f"Total files: {len(midi_files)}")
        print(f"Successful: {success_count}")
        print(f"Failed: {failed_count}")
        print(f"Output directory: {output_dir}")
        
        if failed_files:
            print(f"\nFailed files:")
            for f in failed_files[:10]:  # Show first 10
                print(f"  - {f}")
            if len(failed_files) > 10:
                print(f"  ... and {len(failed_files) - 10} more")
    
    return {
        'total': len(midi_files),
        'success': success_count,
        'failed': failed_count,
        'pattern': pattern_str,
        'output_dir': str(output_dir),
        'chromatic_mapping': chromatic_mapping,
        'failed_files': failed_files
    }

## Run Data Augmentation

### Pattern: Pythagorean-like [9, 9, 4, 9, 9, 9, 4]

This pattern uses whole tones of 9 steps (≈ 203.77 cents) and semitones of 4 steps (≈ 90.57 cents), approximating Pythagorean tuning.

In [7]:
# Pythagorean-like pattern
pythagorean_pattern = [9, 9, 4, 9, 9, 9, 4]

# Check if input directory exists and has files
if INPUT_DIR.exists():
    files = list(INPUT_DIR.glob('*.mid'))
    print(f"Input directory exists: {INPUT_DIR}")
    print(f"Found {len(files)} MIDI files")
    if files:
        print(f"\nFirst 5 files:")
        for f in files[:5]:
            print(f"  - {f.name}")
else:
    print(f"Input directory does not exist: {INPUT_DIR}")
    print("Please ensure the directory contains your 12-TET MIDI files.")

Input directory exists: ../dataset/midi_files/mpe
Found 48062 MIDI files

First 5 files:
  - 44372_Limbo_Ab_major.mid
  - 29335_La Vem Você - AB AC AB A_G_major.mid
  - 37263_Speevy_Eb_major.mid
  - 03671_Boca Sem Dente  - AAB AB_B_major.mid
  - 32439_Vai Saudade 1_Eb_major.mid


In [8]:
# Run the conversion with Pythagorean pattern
results = process_dataset_with_pattern(
    pattern=pythagorean_pattern,
    input_dir=INPUT_DIR,
    output_base_dir=OUTPUT_BASE_DIR,
    sharp_bias=0.5,  # Place accidentals in the middle of intervals
    verbose=True
)

53-EDO Data Augmentation
Pattern: [9, 9, 4, 9, 9, 9, 4]
Pattern string: 9-9-4-9-9-9-4
Input directory: ../dataset/midi_files/mpe
Output directory: ../dataset/midi_files/mpe53/9-9-4-9-9-9-4
Sharp bias: 0.5

Chromatic mapping (pitch class → 53-EDO step):
   C: step  0 (+0.0 cents from 12-TET)
  C#: step  4 (-9.4 cents from 12-TET)
   D: step  9 (+3.8 cents from 12-TET)
  D#: step 13 (-5.7 cents from 12-TET)
   E: step 18 (+7.5 cents from 12-TET)
   F: step 22 (-1.9 cents from 12-TET)
  F#: step 26 (-11.3 cents from 12-TET)
   G: step 31 (+1.9 cents from 12-TET)
  G#: step 35 (-7.5 cents from 12-TET)
   A: step 40 (+5.7 cents from 12-TET)
  A#: step 44 (-3.8 cents from 12-TET)
   B: step 49 (+9.4 cents from 12-TET)

Found 48062 MIDI files to process

Processed 100/48062 files...
Processed 200/48062 files...
Processed 300/48062 files...
Processed 400/48062 files...
Processed 500/48062 files...
Processed 600/48062 files...
Processed 700/48062 files...
Processed 800/48062 files...
Processed 

## Alternative Patterns

Here are some other interesting 53-EDO patterns you can try:

In [8]:
# Dictionary of named patterns
NAMED_PATTERNS = {
    'pythagorean': [9, 9, 4, 9, 9, 9, 4],      # Pure fifths approximation
    'just_major': [9, 8, 5, 9, 8, 9, 5],       # 5-limit just intonation major
    'meantone': [8, 8, 6, 8, 8, 8, 7],         # Quarter-comma meantone approximation
    'equal_53': [8, 8, 5, 8, 8, 9, 7],         # Balanced/equal-ish distribution
    'septimal': [9, 7, 6, 9, 8, 8, 6],         # 7-limit influenced
}

print("Available patterns:")
print("-" * 60)
for name, pattern in NAMED_PATTERNS.items():
    print(f"{name:>15}: {pattern} (sum={sum(pattern)})")
    
# Verify all patterns sum to 53
print("\nValidation:")
for name, pattern in NAMED_PATTERNS.items():
    valid = "✓" if sum(pattern) == 53 and len(pattern) == 7 else "✗"
    print(f"  {name}: {valid}")

Available patterns:
------------------------------------------------------------
    pythagorean: [9, 9, 4, 9, 9, 9, 4] (sum=53)
     just_major: [9, 8, 5, 9, 8, 9, 5] (sum=53)
       meantone: [8, 8, 6, 8, 8, 8, 7] (sum=53)
       equal_53: [8, 8, 5, 8, 8, 9, 7] (sum=53)
       septimal: [9, 7, 6, 9, 8, 8, 6] (sum=53)

Validation:
  pythagorean: ✓
  just_major: ✓
  meantone: ✓
  equal_53: ✓
  septimal: ✓


In [9]:
def run_multiple_patterns(patterns: Dict[str, List[int]], skip_existing: bool = True):
    """
    Run data augmentation with multiple patterns.
    
    Args:
        patterns: Dictionary of pattern_name -> pattern_list
        skip_existing: Skip patterns that already have output directories
    """
    all_results = {}
    
    for name, pattern in patterns.items():
        pattern_str = '-'.join(map(str, pattern))
        output_dir = OUTPUT_BASE_DIR / pattern_str
        
        if skip_existing and output_dir.exists() and list(output_dir.glob('*.mid')):
            print(f"\n⏭️  Skipping '{name}' - output directory already exists with files")
            continue
        
        print(f"\n🎵 Processing pattern: {name}")
        results = process_dataset_with_pattern(
            pattern=pattern,
            verbose=True
        )
        all_results[name] = results
    
    return all_results

In [10]:
# Uncomment to run all patterns:
all_results = run_multiple_patterns(NAMED_PATTERNS, skip_existing=True)


⏭️  Skipping 'pythagorean' - output directory already exists with files

🎵 Processing pattern: just_major
53-EDO Data Augmentation
Pattern: [9, 8, 5, 9, 8, 9, 5]
Pattern string: 9-8-5-9-8-9-5
Input directory: ../dataset/midi_files/mpe
Output directory: ../dataset/midi_files/mpe53/9-8-5-9-8-9-5
Sharp bias: 0.5

Chromatic mapping (pitch class → 53-EDO step):
   C: step  0 (+0.0 cents from 12-TET)
  C#: step  4 (-9.4 cents from 12-TET)
   D: step  9 (+3.8 cents from 12-TET)
  D#: step 13 (-5.7 cents from 12-TET)
   E: step 17 (-15.1 cents from 12-TET)
   F: step 22 (-1.9 cents from 12-TET)
  F#: step 26 (-11.3 cents from 12-TET)
   G: step 31 (+1.9 cents from 12-TET)
  G#: step 35 (-7.5 cents from 12-TET)
   A: step 39 (-17.0 cents from 12-TET)
  A#: step 43 (-26.4 cents from 12-TET)
   B: step 48 (-13.2 cents from 12-TET)

Found 48062 MIDI files to process

Processed 100/48062 files...
Processed 200/48062 files...
Processed 300/48062 files...
Processed 400/48062 files...
Processed 500/4

## Verify Output

Let's verify that the output files were created correctly.

In [ ]:
def verify_output(pattern: List[int]):
    """Verify output files for a given pattern."""
    pattern_str = '-'.join(map(str, pattern))
    output_dir = OUTPUT_BASE_DIR / pattern_str
    
    if not output_dir.exists():
        print(f"Output directory does not exist: {output_dir}")
        return
    
    files = list(output_dir.glob('*.mid'))
    print(f"Output directory: {output_dir}")
    print(f"Files created: {len(files)}")
    
    if files:
        print(f"\nFirst 5 output files:")
        for f in sorted(files)[:5]:
            size_kb = f.stat().st_size / 1024
            print(f"  - {f.name} ({size_kb:.1f} KB)")
        
        # Analyze one file
        print(f"\n📊 Analyzing first file...")
        sample_file = sorted(files)[0]
        mid = MidiFile(sample_file)
        
        # Count message types
        msg_counts = {}
        pitch_bends = []
        
        for track in mid.tracks:
            for msg in track:
                if not msg.is_meta:
                    msg_counts[msg.type] = msg_counts.get(msg.type, 0) + 1
                    if msg.type == 'pitchwheel':
                        pitch_bends.append(msg.pitch)
        
        print(f"  Message types: {msg_counts}")
        print(f"  Duration: {mid.length:.1f} seconds")
        
        if pitch_bends:
            unique_bends = set(pitch_bends)
            print(f"  Unique pitch bend values: {len(unique_bends)}")
            print(f"  Pitch bend range: {min(pitch_bends)} to {max(pitch_bends)}")

# Verify Pythagorean pattern output
verify_output(pythagorean_pattern)

## Comparison: Cents Deviation Table

This table shows how each pitch class deviates from 12-TET in cents for different patterns.

In [ ]:
import pandas as pd

def create_cents_comparison_table(patterns: Dict[str, List[int]]) -> pd.DataFrame:
    """Create a table comparing cents deviations for different patterns."""
    data = {'Pitch Class': pc_names}
    
    for name, pattern in patterns.items():
        mapping = interpolate_chromatic_mapping(pattern)
        cents_devs = []
        for pc in range(12):
            cents_12tet = pc * 100
            cents_53edo = mapping[pc] * CENTS_PER_STEP
            cents_devs.append(round(cents_53edo - cents_12tet, 1))
        data[name] = cents_devs
    
    return pd.DataFrame(data)

# Create comparison table
comparison_df = create_cents_comparison_table(NAMED_PATTERNS)
print("Cents deviation from 12-TET for each pattern:")
print(comparison_df.to_string(index=False))

## Custom Pattern Generator

Use this cell to create and test your own custom patterns.

In [ ]:
def analyze_pattern(pattern: List[int], name: str = "Custom"):
    """Analyze a 53-EDO pattern and show its properties."""
    if len(pattern) != 7:
        print(f"❌ Pattern must have 7 elements (got {len(pattern)})")
        return
    
    total = sum(pattern)
    if total != 53:
        print(f"❌ Pattern must sum to 53 (got {total})")
        print(f"   Difference: {53 - total} (add this to make valid)")
        return
    
    print(f"✅ Pattern '{name}' is valid!")
    print(f"   {pattern}")
    print()
    
    # Intervals in cents
    interval_names = ['C-D', 'D-E', 'E-F', 'F-G', 'G-A', 'A-B', 'B-C']
    print("Intervals:")
    for i, (name_i, steps) in enumerate(zip(interval_names, pattern)):
        cents = steps * CENTS_PER_STEP
        print(f"  {name_i}: {steps} steps = {cents:.1f} cents")
    
    print()
    
    # Chromatic mapping
    mapping = interpolate_chromatic_mapping(pattern)
    print("Chromatic mapping:")
    for pc in range(12):
        step = mapping[pc]
        cents_53 = step * CENTS_PER_STEP
        cents_12 = pc * 100
        diff = cents_53 - cents_12
        print(f"  {pc_names[pc]:>2}: step {step:>2} = {cents_53:>6.1f}¢ ({diff:+5.1f}¢ from 12-TET)")

# Example: Analyze the Pythagorean pattern
analyze_pattern([9, 9, 4, 9, 9, 9, 4], "Pythagorean")

In [ ]:
# Create and run with a custom pattern
# Modify this pattern to experiment with different tunings

custom_pattern = [9, 8, 5, 9, 8, 9, 5]  # Just intonation major

analyze_pattern(custom_pattern, "Custom")

# Uncomment to run conversion with custom pattern:
# results = process_dataset_with_pattern(custom_pattern, verbose=True)